<a href="https://colab.research.google.com/github/Madushani-Weerasekara/D10_Fake-News-Classifier-Using-LSTM/blob/main/D10_Fake_News_Classifier_Using_LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Fake News Classifier Using LSTM

In [37]:
import pandas as pd


In [38]:
df = pd.read_csv('/content/FakeNewsNet.csv')

In [39]:
df.head()

,title,news_url,source_domain,tweet_num,real
0,Kandi Burruss Explodes Over Rape Accusation on...,http://toofab.com/2017/05/08/real-housewives-a...,toofab.com,42,1
1,People's Choice Awards 2018: The best red carp...,https://www.today.com/style/see-people-s-choic...,www.today.com,0,1
2,Sophia Bush Sends Sweet Birthday Message to 'O...,https://www.etonline.com/news/220806_sophia_bu...,www.etonline.com,63,1
3,Colombian singer Maluma sparks rumours of inap...,https://www.dailymail.co.uk/news/article-33655...,www.dailymail.co.uk,20,1
4,Gossip Girl 10 Years Later: How Upper East Sid...,https://www.zerchoo.com/entertainment/gossip-g...,www.zerchoo.com,38,1


In [40]:
df.shape

(23196, 5)

In [41]:
df.isnull().sum()

title              0
news_url         330
source_domain    330
tweet_num          0
real               0
dtype: int64

In [42]:
##Drop Null values
df.dropna()

,title,news_url,source_domain,tweet_num,real
0,Kandi Burruss Explodes Over Rape Accusation on...,http://toofab.com/2017/05/08/real-housewives-a...,toofab.com,42,1
1,People's Choice Awards 2018: The best red carp...,https://www.today.com/style/see-people-s-choic...,www.today.com,0,1
2,Sophia Bush Sends Sweet Birthday Message to 'O...,https://www.etonline.com/news/220806_sophia_bu...,www.etonline.com,63,1
3,Colombian singer Maluma sparks rumours of inap...,https://www.dailymail.co.uk/news/article-33655...,www.dailymail.co.uk,20,1
4,Gossip Girl 10 Years Later: How Upper East Sid...,https://www.zerchoo.com/entertainment/gossip-g...,www.zerchoo.com,38,1
...,...,...,...,...,...
23191,Pippa Middleton wedding: In case you missed it...,https://www.express.co.uk/news/royal/807049/pi...,www.express.co.uk,52,1
23192,Zayn Malik & Gigi Hadid’s Shocking Split: Why ...,hollywoodlife.com/2018/03/13/zayn-malik-gigi-h...,hollywoodlife.com,7,0
23193,Jessica Chastain Recalls the Moment Her Mother...,http://www.justjared.com/2018/01/17/jessica-ch...,www.justjared.com,26,1
23194,"Tristan Thompson Feels ""Dumped"" After Khloé Ka...",www.intouchweekly.com/posts/tristan-thompson-f...,www.intouchweekly.com,24,0


In [43]:
# Get the independent feature
x = df.drop('real', axis=1)

In [44]:
#Get the dependent feature
y=df['real']

In [45]:
x.shape

(23196, 4)

In [46]:
y.shape

(23196,)

In [47]:
!pip install tensorflow

In [48]:
import tensorflow as tf

In [49]:
tf.__version__

'2.15.0'

In [50]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense


In [51]:
##Vocabulary size
voc_size = 5000

##Onehot Representation

In [52]:
messages = x.copy()

In [53]:
# title of the 1st index
messages['title'][1]

"People's Choice Awards 2018: The best red carpet looks"

In [54]:
messages

,title,news_url,source_domain,tweet_num
0,Kandi Burruss Explodes Over Rape Accusation on...,http://toofab.com/2017/05/08/real-housewives-a...,toofab.com,42
1,People's Choice Awards 2018: The best red carp...,https://www.today.com/style/see-people-s-choic...,www.today.com,0
2,Sophia Bush Sends Sweet Birthday Message to 'O...,https://www.etonline.com/news/220806_sophia_bu...,www.etonline.com,63
3,Colombian singer Maluma sparks rumours of inap...,https://www.dailymail.co.uk/news/article-33655...,www.dailymail.co.uk,20
4,Gossip Girl 10 Years Later: How Upper East Sid...,https://www.zerchoo.com/entertainment/gossip-g...,www.zerchoo.com,38
...,...,...,...,...
23191,Pippa Middleton wedding: In case you missed it...,https://www.express.co.uk/news/royal/807049/pi...,www.express.co.uk,52
23192,Zayn Malik & Gigi Hadid’s Shocking Split: Why ...,hollywoodlife.com/2018/03/13/zayn-malik-gigi-h...,hollywoodlife.com,7
23193,Jessica Chastain Recalls the Moment Her Mother...,http://www.justjared.com/2018/01/17/jessica-ch...,www.justjared.com,26
23194,"Tristan Thompson Feels ""Dumped"" After Khloé Ka...",www.intouchweekly.com/posts/tristan-thompson-f...,www.intouchweekly.com,24


In [55]:
messages.reset_index(inplace=True)

In [56]:
messages

,index,title,news_url,source_domain,tweet_num
0,0,Kandi Burruss Explodes Over Rape Accusation on...,http://toofab.com/2017/05/08/real-housewives-a...,toofab.com,42
1,1,People's Choice Awards 2018: The best red carp...,https://www.today.com/style/see-people-s-choic...,www.today.com,0
2,2,Sophia Bush Sends Sweet Birthday Message to 'O...,https://www.etonline.com/news/220806_sophia_bu...,www.etonline.com,63
3,3,Colombian singer Maluma sparks rumours of inap...,https://www.dailymail.co.uk/news/article-33655...,www.dailymail.co.uk,20
4,4,Gossip Girl 10 Years Later: How Upper East Sid...,https://www.zerchoo.com/entertainment/gossip-g...,www.zerchoo.com,38
...,...,...,...,...,...
23191,23191,Pippa Middleton wedding: In case you missed it...,https://www.express.co.uk/news/royal/807049/pi...,www.express.co.uk,52
23192,23192,Zayn Malik & Gigi Hadid’s Shocking Split: Why ...,hollywoodlife.com/2018/03/13/zayn-malik-gigi-h...,hollywoodlife.com,7
23193,23193,Jessica Chastain Recalls the Moment Her Mother...,http://www.justjared.com/2018/01/17/jessica-ch...,www.justjared.com,26
23194,23194,"Tristan Thompson Feels ""Dumped"" After Khloé Ka...",www.intouchweekly.com/posts/tristan-thompson-f...,www.intouchweekly.com,24


In [57]:
messages.shape

(23196, 5)

In [58]:
import nltk
import re
from nltk.corpus import stopwords

In [59]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [60]:
#Dataset Preprocessing
from nltk.stem import PorterStemmer # Stemming purpose
ps = PorterStemmer()
corpus = []
for i in range(0, len(messages)):
  review = re.sub('[^a-zA-Z]', ' ', messages['title'][i])
  review = review.lower()
  review = review.split()

  review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
  review = ' '.join(review)
  corpus.append(review)

In [61]:
corpus

['kandi burruss explod rape accus real housew atlanta reunion video',
 'peopl choic award best red carpet look',
 'sophia bush send sweet birthday messag one tree hill co star hilari burton breyton eva',
 'colombian singer maluma spark rumour inappropri relationship aunt',
 'gossip girl year later upper east sider shock world chang pop cultur forev',
 'gwen stefani got dump blake shelton jealousi drama exclus',
 'broward counti sheriff fire lie parkland',
 'amber rose shut french montana date rumor call rapper bruvaaa',
 'mindi kale make first post babi appear disneyland wrinkl time co star',
 'katharin mcphee butcher toni nomin drink',
 'wag miami star ashley nicol robert philip wheeler marri',
 'mel gibson hollywood pedophil nowher left hide',
 'medium tyler henri address chill messag kristin cavallari deceas brother express read',
 'dwt season result week disney night',
 'via cq subscrib tax foundat',
 'reason tarek el moussa overcom latest back injuri',
 'david cassidi cut estrang 

In [62]:
corpus[0]

'kandi burruss explod rape accus real housew atlanta reunion video'

In [63]:
onehot_rep=[one_hot(words,voc_size)for words in corpus]
onehot_rep

[[94, 1919, 1895, 1910, 615, 3089, 183, 1830, 4548, 3912],
 [3408, 3384, 4642, 3661, 3354, 3846, 1403],
 [1445,
  4137,
  3123,
  192,
  2796,
  4142,
  2530,
  4677,
  3359,
  1593,
  4557,
  3368,
  511,
  4521,
  4107],
 [2849, 3935, 1977, 1232, 2353, 3447, 4772, 1976],
 [1806, 4847, 854, 4463, 2393, 3938, 1932, 4222, 237, 3896, 169, 4194, 886],
 [4662, 611, 3260, 680, 3063, 3208, 2198, 2087, 305],
 [1508, 1029, 1088, 2390, 3906, 3204],
 [2120, 737, 3810, 1882, 4311, 4209, 3862, 2213, 1279, 3027],
 [4304, 3178, 2984, 3813, 1435, 4013, 988, 2098, 4671, 3688, 1593, 4557],
 [2725, 3528, 139, 2178, 722, 1974],
 [3296, 2051, 4557, 573, 4553, 2761, 725, 1351, 1130],
 [4466, 1774, 2660, 4749, 4613, 994, 375],
 [2951, 2713, 1729, 683, 1162, 4142, 56, 3476, 1693, 3148, 2707, 1039],
 [391, 4239, 4760, 2104, 1376, 4649],
 [337, 3217, 946, 316, 3977],
 [3075, 3338, 2445, 791, 1808, 1616, 4824, 3901],
 [3183, 1045, 3373, 4733, 1355, 2132, 3932, 1754, 1592, 298],
 [3964, 3874, 3047, 1050, 4, 4111

In [64]:
onehot_rep[0]

[94, 1919, 1895, 1910, 615, 3089, 183, 1830, 4548, 3912]

##Embedding Representation

In [65]:
sent_length = 20 #Max length of he sentences
embedded_docs = pad_sequences(onehot_rep, padding = 'pre', maxlen = sent_length)
print(embedded_docs)

[[   0    0    0 ... 1830 4548 3912]
 [   0    0    0 ... 3354 3846 1403]
 [   0    0    0 ...  511 4521 4107]
 ...
 [   0    0    0 ... 2501 2887 2241]
 [   0    0    0 ... 1357  103  305]
 [   0    0    0 ... 2118 3246 4642]]


In [66]:
print(embedded_docs[0])

[   0    0    0    0    0    0    0    0    0    0   94 1919 1895 1910
  615 3089  183 1830 4548 3912]


In [67]:
print(embedded_docs[1])

[   0    0    0    0    0    0    0    0    0    0    0    0    0 3408
 3384 4642 3661 3354 3846 1403]


In [68]:
#Creating model
embedding_vector_features = 40 #Features representation
model = Sequential()
model.add(Embedding(voc_size, embedding_vector_features, input_length = sent_length)) #Add the embedding layer
model.add(LSTM(100)) #Add LSTM with 100 nuerons
model.add(Dense(1, activation = 'sigmoid'))
model.compile(loss ='binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
print(model.summary())

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 20, 40)            200000    
                                                                 
 lstm_1 (LSTM)               (None, 100)               56400     
                                                                 
 dense_1 (Dense)             (None, 1)                 101       
                                                                 
Total params: 256501 (1001.96 KB)
Trainable params: 256501 (1001.96 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None


In [69]:
len(embedded_docs), y.shape

(23196, (23196,))

In [70]:
import numpy as np
x_final = np.array(embedded_docs)
y_final = np.array(y)

In [71]:
x_final.shape, y_final.shape


((23196, 20), (23196,))

In [72]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x_final, y_final, test_size=0.33, random_state=42 )


##Model Training

In [73]:
model.fit(x_train, y_train, validation_data=(x_test, y_test),epochs = 10, batch_size= 64)

Epoch 1/10
243/243 [==============================] - 31s 115ms/step - loss: 0.4700 - accuracy: 0.7925 - val_loss: 0.3998 - val_accuracy: 0.8291
Epoch 2/10
243/243 [==============================] - 12s 51ms/step - loss: 0.3465 - accuracy: 0.8502 - val_loss: 0.4013 - val_accuracy: 0.8274
Epoch 3/10
243/243 [==============================] - 13s 52ms/step - loss: 0.3058 - accuracy: 0.8723 - val_loss: 0.4134 - val_accuracy: 0.8247
Epoch 4/10
243/243 [==============================] - 11s 47ms/step - loss: 0.2721 - accuracy: 0.8885 - val_loss: 0.4419 - val_accuracy: 0.8234
Epoch 5/10
243/243 [==============================] - 11s 43ms/step - loss: 0.2376 - accuracy: 0.9046 - val_loss: 0.4815 - val_accuracy: 0.8125
Epoch 6/10
243/243 [==============================] - 12s 48ms/step - loss: 0.2088 - accuracy: 0.9154 - val_loss: 0.5062 - val_accuracy: 0.8086
Epoch 7/10
243/243 [==============================] - 12s 51ms/step - loss: 0.1837 - accuracy: 0.9274 - val_loss: 0.5937 - val_accuracy

##Adding Dropout

In [75]:
#Dropout metrics is for the improve the performance of the model.
from tensorflow.keras.layers import Dropout
#Creating Model
embedding_vector_features =40
model = Sequential()
model.add(Embedding(voc_size, embedding_vector_features, input_length = sent_length))
model.add(Dropout(0.3))
model.add(LSTM(100))
model.add(Dropout(0.3))
model.add(Dense(1, activation = 'sigmoid'))
model.compile(loss = 'crossentropy', optimizer = 'adam', metrics = ['accuracy'])


##Performance Metrics and Accuracy

In [76]:
y_pred = model.predict(x_test)

240/240 [==============================] - 3s 9ms/step


In [77]:
y_pred = np.where(y_pred>0.6, 1,0) # If don't want to keep a treashold of  0.6 then use AUC ROC curve

In [78]:
from sklearn.metrics import confusion_matrix

In [79]:
confusion_matrix(y_test, y_pred)

array([[1854,    0],
       [5801,    0]])

In [80]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test, y_pred)

0.24219464402351404

In [81]:
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.24      1.00      0.39      1854
           1       0.00      0.00      0.00      5801

    accuracy                           0.24      7655
   macro avg       0.12      0.50      0.19      7655
weighted avg       0.06      0.24      0.09      7655



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
